# Overview Materi

Source: https://www.youtube.com/watch?v=LDRbO9a6XPU

Jelaskan secara singkat apa itu decision tree menurut pemahamanmu!

=> Decision trees adalah algoritme pembelajaran yang diawasi dan bersifat non-parametrik, yang digunakan untuk tugas klasifikasi dan regresi.

# Import Data & Libraries

In [ ]:
from __future__ import print_function

# label kolom
header = ["color", "diameter", "label"]

# data training
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# data testing
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# Fungsi Dasar

In [ ]:
# fungsi mencari apa saja unique value dari suatu kolom
def unique_vals(rows, col):
    uniq = []
    for r in rows:
        v = r[col]
        if v not in uniq:
            uniq.append(v)
    return uniq

def class_counts(rows):
    counts = {}
    for r in rows:
        lbl = r[-1]
        if lbl not in counts:
            counts[lbl] = 0
        counts[lbl] += 1
    return counts

def is_numeric(v):
    return type(v) in (int, float)

# contoh penggunaan
COL_COLOR = 0
COL_DIAMETER = 1
COL_LABEL = 2

print("Unique color (train):   ", unique_vals(training_data, COL_COLOR))
print("Unique diameter (train):", unique_vals(training_data, COL_DIAMETER))
print("Class counts (train):   ", class_counts(training_data))

Unique color (train):    ['Green', 'Yellow', 'Red']
Unique diameter (train): [3, 1]
Class counts (train):    {'Apple': 2, 'Grape': 2, 'Lemon': 1}


In [ ]:
# fungsi Menghitung jumlah unique value dari suatu kolom
def class_counts(rows):
    counts = {}
    for r in rows:
        lbl = r[-1]
        if lbl not in counts:
            counts[lbl] = 0
        counts[lbl] += 1
    return counts

# contoh penggunaan
COL_COLOR = 0
COL_DIAMETER = 1
COL_LABEL = 2

print("n_unique diameter:", len(unique_vals(training_data, COL_DIAMETER)))
print("n_unique label:",    len(unique_vals(training_data, COL_LABEL)))

n_unique diameter: 2
n_unique label: 3


In [ ]:
# fungsi pengecekan suatu value numerik atau bukan
def is_numeric(value):
  return (type(value) is int) or (type(value) is float)

# contoh penggunaan
print(is_numeric(3))
print(is_numeric(3.5))
print(is_numeric("3"))
print(is_numeric("Green"))

print(is_numeric(training_data[0][1]))
print(is_numeric(training_data[0][0]))

True
True
False
False
True
False


In [ ]:
# kelas untuk merepresentasikan pertanyaan pada decision tree
class Question:

    # inisialisasi kolom dan nilai pertanyaan
    def __init__(self, column, value):
        self.column = column
        self.value = value

    # mengecek apakah contoh data sesuai dengan pertanyaan
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    # menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):
        cond = ">=" if is_numeric(self.value) else "=="
        return f"Is {header[self.column]} {cond} {self.value}?"

# contoh penggunaan
q1 = Question(1, 3)
q2 = Question(0, "Red")

print(q1)
print([q1.match(r) for r in training_data])

print(q2)
print([q2.match(r) for r in training_data])

Is diameter >= 3?
[True, True, False, False, True]
Is color == Red?
[False, False, True, True, False]


In [ ]:
# membagi dataset menjadi dua berdasarkan pertanyaan
def partition(rows, question):
    true_rows, false_rows = [], []
    for r in rows:
        if question.match(r):
            true_rows.append(r)
        else:
            false_rows.append(r)
    return true_rows, false_rows

# contoh penggunaan
q = Question(1, 3)
true_rows, false_rows = partition(training_data, q)

print(q)
print("True rows:", true_rows)
print("False rows:", false_rows)
print("Jumlah True:", len(true_rows), "| Jumlah False:", len(false_rows))

Is diameter >= 3?
True rows: [['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]
False rows: [['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
Jumlah True: 3 | Jumlah False: 2


**apa itu gini impurity?**
<br> gini impurity berfungsi mengukur tingkat ketidakmurnian atau ketidakteraturan pada sebuah simpul (node) dalam pohon

In [ ]:
# menghitung nilai Gini Impurity untuk sebuah dataset
def gini(rows):
    n = len(rows)
    if n == 0:
        return 0.0

    counts = {}
    for r in rows:
        lbl = r[-1]
        counts[lbl] = counts.get(lbl, 0) + 1

    impurity = 1.0
    for lbl in counts:
        p = counts[lbl] / float(n)
        impurity -= p * p
    return impurity

# contoh penggunaan
print("Gini(training):", gini(training_data))


q = Question(1, 3)
true_rows, false_rows = partition(training_data, q)

print(q)
print("Gini(true_rows): ", gini(true_rows))
print("Gini(false_rows):", gini(false_rows))

Gini(training): 0.6399999999999999
Is diameter >= 3?
Gini(true_rows):  0.4444444444444445
Gini(false_rows): 0.0


**apa itu information gain?**
<br> information gain berfungsi mengukur seberapa efektif sebuah fitur dalam memisahkan data berdasarkan kelas-kelasnya

In [ ]:
# menghitung nilai Information Gain dari pemisahan dataset
def info_gain(left, right, current_uncertainty):

    total = len(left) + len(right)
    if total == 0:
        return 0.0

    p = len(left) / float(total)
    gain = current_uncertainty - (p * gini(left) + (1 - p) * gini(right))
    return gain

# contoh penggunaan
parent_gini = gini(training_data)
print("Gini parent (training):", parent_gini)

q1 = Question(1, 3)
true_rows, false_rows = partition(training_data, q1)
gain1 = info_gain(true_rows, false_rows, parent_gini)

print(q1)
print("Gini(true):", gini(true_rows), "| Gini(false):", gini(false_rows))
print("Information Gain (diameter>=3):", gain1)

q2 = Question(0, "Red")
t2, f2 = partition(training_data, q2)
gain2 = info_gain(t2, f2, parent_gini)

print(q2)
print("Gini(true):", gini(t2), "| Gini(false):", gini(f2))
print("Information Gain (color=='Red'):", gain2)

Gini parent (training): 0.6399999999999999
Is diameter >= 3?
Gini(true): 0.4444444444444445 | Gini(false): 0.0
Information Gain (diameter>=3): 0.37333333333333324
Is color == Red?
Gini(true): 0.0 | Gini(false): 0.4444444444444445
Information Gain (color=='Red'): 0.37333333333333324


In [ ]:
# mencari pertanyaan terbaik untuk membagi dataset berdasarkan information gain tertinggi
def find_best_split(rows):
    best_gain = 0.0
    best_question = None
    current_uncertainty = gini(rows)

    n_features = len(rows[0]) - 1

    for col in range(n_features):
        values = unique_vals(rows, col)
        for val in values:
            q = Question(col, val)

            # splitting the dataset
            left, right = partition(rows, q)

            if len(left) == 0 or len(right) == 0:
                continue

            gain = info_gain(left, right, current_uncertainty)

            if gain > best_gain:
                best_gain = gain
                best_question = q

    return best_gain, best_question

# contoh penggunaan
gain, question = find_best_split(training_data)
print("Best gain:", gain)
print("Best question:", question)

if question is not None:
    left, right = partition(training_data, question)
    print("Left rows :", left)
    print("Right rows:", right)
else:
    print("Tidak ada split yang membagi data.")


Best gain: 0.37333333333333324
Best question: Is color == Red?
Left rows : [['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
Right rows: [['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]


# Fungsi Decision Tree

In [ ]:
# merepresentasikan node daun (leaf) pada decision tree yang berisi hasil prediksi
class Leaf:

    # inisialisasi leaf dengan menghitung jumlah kemunculan tiap kelas
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [ ]:
# merepresentasikan node keputusan (decision node) yang berisi pertanyaan dan cabang
class Decision_Node:

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [ ]:
# membangun decision tree secara rekursif
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)

    left, right = partition(rows, question)

    true_branch = build_tree(left)
    false_branch = build_tree(right)
    return Decision_Node(question, true_branch, false_branch)

In [ ]:
# mencetak struktur decision tree secara rekursif dalam format teks
def print_tree(node, spacing=""):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        print(spacing + "Predict", dict(node.predictions))
        return

    # mencetak pertanyaan pada node saat ini
    print(spacing + str(node.question))

    # mencetak cabang true secara rekursif
    print(spacing + "--> True:")
    print_tree(node.true_branch, spacing + "  ")

    # mencetak cabang false secara rekursif
    print(spacing + "--> False:")
    print_tree(node.false_branch, spacing + "  ")

# contoh penggunaan
my_tree = build_tree(training_data)
print_tree(my_tree)

Is color == Red?
--> True:
  Predict {'Grape': 2}
--> False:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}


In [ ]:
# mengklasifikasikan satu baris data menggunakan decision tree
def classify(row, node):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        return node.predictions

    # menentukan apakah mengikuti cabang true atau cabang false
    # dengan membandingkan nilai fitur pada baris dengan pertanyaan di node
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

# contoh penggunaan
row = testing_data[0]
dist = classify(row, my_tree)
pred = max(dist, key=dist.get)
print(row, "=>", print_leaf(dist), "| pred:", pred, "| true:", row[-1])

correct = 0
for r in testing_data:
    d = classify(r, my_tree)
    p = max(d, key=d.get)
    correct += (p == r[-1])
    print(r, "=>", print_leaf(d), "| pred:", p, "| true:", r[-1])
print("Accuracy:", correct / len(testing_data))

['Green', 3, 'Apple'] => {'Apple': '100%'} | pred: Apple | true: Apple
['Green', 3, 'Apple'] => {'Apple': '100%'} | pred: Apple | true: Apple
['Yellow', 4, 'Apple'] => {'Apple': '50%', 'Lemon': '50%'} | pred: Apple | true: Apple
['Red', 2, 'Grape'] => {'Grape': '100%'} | pred: Grape | true: Grape
['Red', 1, 'Grape'] => {'Grape': '100%'} | pred: Grape | true: Grape
['Yellow', 3, 'Lemon'] => {'Apple': '50%', 'Lemon': '50%'} | pred: Apple | true: Lemon
Accuracy: 0.8


In [ ]:
# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    total = sum(counts.values())
    probs = {}
    for lbl, cnt in counts.items():
        persen = int(100 * cnt / total) if total > 0 else 0
        probs[lbl] = f"{persen}%"
    return probs

# contoh penggunaan
print(print_leaf({'Apple': 2, 'Lemon': 1}))

{'Apple': '66%', 'Lemon': '33%'}


# Predict Using Decision Tree

In [ ]:
# menguji decision tree dengan data uji dan membandingkan hasil prediksi dengan label asli
correct = 0
for row in testing_data:
    dist = classify(row, my_tree)
    pred = max(dist, key=dist.get)
    true_label = row[-1]
    ok = (pred == true_label)
    correct += 1 if ok else 0
    print(row, "=>", print_leaf(dist), "| pred:", pred, "| true:", true_label, "|", "OK" if ok else "X")

accuracy = correct / len(testing_data)
print("Accuracy:", accuracy)

['Green', 3, 'Apple'] => {'Apple': '100%'} | pred: Apple | true: Apple | OK
['Yellow', 4, 'Apple'] => {'Apple': '50%', 'Lemon': '50%'} | pred: Apple | true: Apple | OK
['Red', 2, 'Grape'] => {'Grape': '100%'} | pred: Grape | true: Grape | OK
['Red', 1, 'Grape'] => {'Grape': '100%'} | pred: Grape | true: Grape | OK
['Yellow', 3, 'Lemon'] => {'Apple': '50%', 'Lemon': '50%'} | pred: Apple | true: Lemon | X
Accuracy: 0.8
